In [26]:
import numpy as np
import pandas as pd
from statsmodels.sandbox.regression import gmm

In [4]:

# Load the data
dta = pd.read_csv('/Users/veronica/Downloads/euler.csv', parse_dates=[0])
print(dta.head())

         qtr       r       c
0 1960-01-01  0.0435  1585.6
1 1960-04-01  0.0323  1605.1
2 1960-07-01  0.0230  1598.5
3 1960-10-01  0.0230  1600.3
4 1961-01-01  0.0224  1600.2


In [5]:
dta['c_growth'] = dta['c'] / dta['c'].shift(1)
dta['c_growth_lag1'] = dta['c_growth'].shift(1)
dta['r_lag1'] = dta['r'].shift(1)
dta['r_lag2'] = dta['r'].shift(2)
dta['r_forw1'] = dta['r'].shift(-1)
dta['c_lag1'] = dta['c'].shift(1)
dta['c_forw1'] = dta['c'].shift(-1)
dta['const'] = 1

dta_clean = dta.dropna()

endog_df = dta_clean[['r_forw1', 'c_forw1', 'c']]
exog_df = endog_df
instrument_df = dta_clean[['r_lag1', 'r_lag2', 'c_growth', 'c_growth_lag1', 'const']]

endog, exog, instrument = map(np.asarray, [endog_df, exog_df, instrument_df])


In [29]:
def moment_consumption2(params, exog):
    beta, gamma = params
    r_forw1, c_forw1, c = exog.T
    
    # 2nd part of moment condition without instrument   
    predicted = beta * (1. + r_forw1) * np.power(c_forw1 / c, -gamma)
    return predicted

endog2 = np.ones(exog.shape[0])
mod2 = gmm.NonlinearIVGMM(endog2, exog, instrument, moment_consumption2, k_moms=4)
w0inv = np.dot(instrument.T, instrument) / len(endog2)  
res2_hac4_2s = mod2.fit([1,-1], maxiter=2, inv_weights=w0inv, weights_method='hac', wargs={'maxlag':4})

print(res2_hac4_2s.summary(yname='Euler Eq', xname=['discount', 'CRRA']))


Optimization terminated successfully.
         Current function value: 0.000355
         Iterations: 5
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: 0.045800
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
                            NonlinearIVGMM Results                            
Dep. Variable:               Euler Eq   Hansen J:                        8.565
Model:                 NonlinearIVGMM   Prob (Hansen J):                0.0138
Method:                           GMM                                         
Date:                Thu, 29 Feb 2024                                         
Time:                        22:23:22                                         
No. Observations:                 187                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------

In [6]:
beta_estimate = res2_hac4_2s.params[0]
gamma_estimate = res2_hac4_2s.params[1]

print("Estimated beta:", beta_estimate)
print("Estimated gamma:", gamma_estimate)


Estimated beta: 0.9070313482230734
Estimated gamma: -5.091699165431186


In [17]:
hansen_test = res2_hac4_2s.jtest()
J_statistic = hansen_test[0]
critical_value = hansen_test[1]
print("Hansen-Sargan J-statistic:", J_statistic)
print("Prob(Hansen):", critical_value)

Hansen-Sargan J-statistic: 8.564643757532675
Prob(Hansen): 0.013810558388181932
